In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import importlib

import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm

from itertools import product
from sklearn.pipeline import make_pipeline

from criteriaetl.utils.expansion_func import (get_value_counts_with_expansion_factor,
    get_percentage_table_with_expansion_factor)
from criteriaetl.utils.display_func import cdisplay, percentage_count_plot
from criteriaetl.utils.common_func import (get_weighted_complete_randomization_series_on_subset, 
    proportional_cut, weighted_qcut, get_partition_bool_columns_dict)
from criteriaetl.transformers.columns_base import (NameTransformer, 
    ReplaceTransformer, SelectTransformer, AssignTransformer)
from criteriaetl.transformers.rows_base import AggregateTransformer
from criteriaetl.transformers.fusion_base import MergeTransformer

from projectetl.utils.dataload import (load_survey_data_do, save_survey_with_pickle,
                                       load_survey_from_pickle)
from projectetl.utils.config import (ENCFT_SURVEY_PATH, ENCFT_PREVIOUS_SURVEY_PATH,
                                     ENCFT_OBJECT_DIR, INFLATION_OBJECT_DIR, DATA_DIR)

In [ ]:
get_raw = 0 # if 1 will read the raw data, if 0 will load the data from a previously serialized pickle object
year = 2020 # year we want to analyze
month = 6 # month to analyze
trimester = 2 # trimester of the year we want to analyze

In [ ]:
if get_raw:
    survey_raw = load_survey_data_do(ENCFT_SURVEY_PATH)
    save_survey_with_pickle(survey_raw, ENCFT_OBJECT_DIR / 'encft202001-202004.pkl')
else:
    survey_raw = load_survey_from_pickle(ENCFT_OBJECT_DIR / 'encft202001-202004.pkl')

In [ ]:
survey_household_raw = survey_raw['Hogar']
survey_member_raw = survey_raw['Miembros']
survey_house_raw = survey_raw['Vivienda']

In [ ]:
survey_member_raw['miembro'].value_counts()

## Assign Transformer

In [ ]:
equipos = {
    'basicos': ['estufa', 'refrigerador', 'televisor', 'lavadora'],
    'modernos': ['automovil', 'aire_acondicionado', 'computador']
}

In [ ]:
assign_icv_map = {
    **{
        f'tiene_{equipo}_icv2': lambda df, equipo=equipo: df[equipo] == 1
        for equipo in equipos['basicos'] + equipos['modernos']
    },
    **{
        f'cantidad_equipos_{tipo}':
            lambda df, group=group: df[[f'tiene_{equipo}_icv2' for equipo in group]].sum(1)
        for tipo, group in equipos.items()
    },

    'cantidad_miembros': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['miembro'].transform('count'),

    'cantidad_menor_5': lambda df: df['edad'] <= 5, # pobmeno5

    'cantidad_menor_5_promedio': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['cantidad_menor_5'].transform('mean') # pobmen5h
}

assign_icv_transformer = AssignTransformer(assign_icv_map)
survey_icv_assigned = assign_icv_transformer.transform(survey_member_raw)

In [ ]:
survey_icv_assigned[['cantidad_miembros']]

## Select Transformers

In [ ]:
select_wall_material_map = {
    'material_pared_exterior_icv2': {
        lambda df: df['material_pared_exterior_esp'] == "ARENA CEMENTO ALAMBRE Y MADERA": 4,
        lambda df: df['material_pared_exterior_esp'] == "ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "ARENA Y VARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "ARENA, CEMENTO, ALAMABRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "ARENA, CEMENTO, ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "ARENA, VARILA, CEMENTO Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "ARENA, VARILLA, CEMENTO Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "BACIADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "BACIADO DE CEMENTO Y ARENAS": 4,
        lambda df: df['material_pared_exterior_esp'] == "BACIADO DE CEMENTO, ARENA Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "BACIADO DE CEMENTO, ARENA, VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "BACIADO DE CEMENTO, VARILLA Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "BAIADO DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "BASIADO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "BLOCK MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "BVACIADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEEMNTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO ARENA SIN VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO SIN VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO, ARENA, ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO, VARILLA, ALAMBRE, ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "COCRETO VACIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO BACIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO NACIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO VACIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO VACIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO VACIANDO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO VARIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETO VASIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "CONCRETRO": 4,
        lambda df: df['material_pared_exterior_esp'] == "MEZCLA DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACAIDO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACAIDO DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADI DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO BLOCK Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMENTO Y ARENA MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMENTO YARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMETO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMNTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CONCRETO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE CEMENTO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE CEMETO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO DE CONCRETO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEDO CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACEDO DE CEMNTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIA DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADIO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO  CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO  DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO  DE CEMENTO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ALAMBRE CEMENTO ARENA, MADERO": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ALAMBRE Y CONCRETO, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ALAMBRE, Y CONCRETO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA ALAMBRE VARILLA CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA ALAMBRE Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA ALAMBRE Y VARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA CEMENTO VARILLA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA VARILLA ALAMBRE Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA VARILLA CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA VARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA VARRILA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA VASILLA CEMENTE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA Y CEMENTO, VARRILLA, ALMABRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA Y CMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA, CEMENTO,": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA, VARILLA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA, VARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA, VARILLA, CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CAMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO ARENA ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y  ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y ARE4NA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO, ALAMBRE, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO, ARENA, ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CON CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CON CEMENTO SIN VARILLAS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CONCRETO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CONCRETO Y ALAMBRE MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CONCRETO Y MADERO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE  ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ALAMBRE CON CEMENTO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ALAMBRE VARILLA Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ALAMBRE Y CONCRETO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ALAMBRES Y CONCRETOS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE AREAN Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA ALAMBRA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA ALAMBRADO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA ALAMBRE Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA CON CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y  CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y ALAMBRE Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y ALAMBRES": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO ALAMBRE Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO SORRILLA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO Y VARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO.": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTOC": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA YCEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, ALAMBRE VARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, ALAMBRE Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, BARILLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO Y BERILLAS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, VARILLA, ALAMBRE":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, LAMBRE Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, VARILLA Y CEMENTO":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, VARILLA, ALAMBRE Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA,VARRILLA Y ALAMBRE":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA. ALAMBRE, CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENAY ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARGRO, GRAVA Y VARILLA CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ATRENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CAMENTO Y ARADO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENETO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTIO Y ARENA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO  ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO , ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO ARENA ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO AY ARENA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO TY ARENA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y AREANA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y ARENA Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y ARENAS":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y MADERA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ALAMBRE Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ALAMBRE, ARENA Y MADERA":	7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA Y BARILLA":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA Y BARILLAS":	4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA, ALAMBRE Y MADERA":	7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA, BARILLAS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA, CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, VARILLA, CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTYO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMNETO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMNETO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CONCRETO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CONCRETO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CONCRETO, ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE DE CONCRETO MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE MEZCLA CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE MEZCLA DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE MEZCLA DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE MEZCLA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE MEZLCA DE CEMENTOS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE TABLA Y PALMA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE VARILLA, ARENA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO EN CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO EN CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO EN CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO MEZCLA CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO MEZCLA DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO MEZCLA DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO MEZCLA DE CEMENTO ARMADO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO MEZCLA DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO MEZLCA ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO Y CEMENTO DE ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO Y CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO Y MADERA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO Y TABLA DE PALMA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, ARENA, VARILLA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, CEMENTO Y ARENAS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, CEMENTO, ARENA Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, CEMENTO, Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO, MEZCLA DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADODE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADOR DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADOR DE MEZCLA CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADOS DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADOS EN CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIANDO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIANDO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIAO MEZCLA DE CEENTI Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIJA BUENA CEMENTO Y MADERA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIOADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIOADO DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIODO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIODO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VAIADO DE CEMENTOS": 4,
        lambda df: df['material_pared_exterior_esp'] == "VARILLA CEMENTO ALAMBRE Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VARILLA,CEMENTO, ARENA Y ALAMBRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VECIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VIACIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "B LOCK Y TABLA DE PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] == "BLCOK Y PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] == "BLOCK PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y  PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y DALMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y PALMA.": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y PALMAS": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y PLAMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y TABLA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y TABLA DE PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y TABLA DE PLAMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"BLOCK Y TABLA PALMA": 9,
        lambda df: df['material_pared_exterior_esp'] ==	"CEMENTO Y PLAYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAJA REBESTIDA DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAJA REBESTIDO DE CEMENTO PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MATA REBESTIDA DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MATA REBISTIDORA DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MATA REVESTIDA CON CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA CON CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA DE REBESTIDO DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIBLE DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA CON CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA DE CEMENTO	": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA DE CEMENTO PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA DE CEMENTO Y PLAYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA DE CEMENTO Y PLEYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDA DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDAS DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REBESTIDO DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVERCIBLE Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVESTIBLE DE CEMENTO Y PLEWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVESTIBLE DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVESTIDA CEMENTO Y PLEWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVESTIDA DE CEMENTO Y PLAYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVESTIDA DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVESTIDA POR CEMENTO Y PLAYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYA REVUELTA": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"MAYOR REVESTIDO CEMENTO Y PLAYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"REBESTIDA DE CEMENTO Y PLYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"VARILLA, ARENA, CEMENTO, ALAMBRE, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] ==	"VECIADO DE CEMENETO Y MADERA": 7,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUERO"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA PRE FABRICACION"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA PRE FABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA PRE FABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA PREFABRICACDA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA PREFABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA PREFABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETA, PRE-FABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAQR PREFABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAR PREDABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAR PREFABRICACDA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAR PREFABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PRE - FABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PRE FABRICACION"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PRE FABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PRE FABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PRE FABRICADO"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PRE- FABRICAADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PREDABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PREFABRICACDAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PREFABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS PREFABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS RPEFABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETAS, PRE FABRICADAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETES"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAQUETES PRE FABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PLAUQETAR PREFABRICADA"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PRE FABRICADA PLAQUETAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PRE FABRICADAS PLAQUETAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PRE FABRICADO PLAQUETAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PRE-FABRICADAS PLAQUETAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PRE-FABRICADO PLAQUETAS"): 4,
        lambda df: (df['material_pared_exterior'] == 99) & (df['material_pared_exterior_esp'] == "PREFABRICADA PLAQUETAS"): 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y AREN": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y PLAYWODD": 7,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y PLAYWOOD": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETA PREFABRICADA": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETAR PREFABRICADA": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETAS PREFABRICACION": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETAS PREFABRICADA": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETON PREFABRICADO": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACEADO CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y ARENA, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA VARILLA CEMENTO MADERA Y ALMABRE": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA VARILLA MADERA Y CEMENTO": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO, VARILLA Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO Y VARILLA, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, VARILLA Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, VARILLA, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, VARILLA, CEMENTO Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENRO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y ARENA Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, VARILLA, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTYO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMNTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE EMENTO Y ARENA.": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DEMENTO ALAMBRE ARENA Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIASDO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VARIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VARIADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO Y VARILLA, MADERA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, VARILLA Y MADERA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO ARENA Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTO Y ATRENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "CEMENTOY ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETA PREFABRICADA": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETE PREFABRICADA": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETON PREFABRICADA": 7,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETON PREFABRICADO": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA ALAMBRE CEMENTO MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA CEMENTO ALAMBRE MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA CEMENTO MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA CEMENTO VARILLA MADERA ALAMBRE": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA MADERA VARILLA CEMENTO ALAMBRE": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA Y CEMENTO, VARILLA, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA, CEMENTO, VARILLA, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO ARENA, CEMENTO, VARILLA, MADERA, ALAMBRE": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTI ARENA ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO VARILLA MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO CEMENTO Y HARINA , ALAMBRE, MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y  CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA Y VARILLA ALAMBRE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE ARENA, CEMENTO, VARILLA, MADERA, ALAMBR": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO ARENA ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y AREN": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO Y VARILLA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTO, ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DE CEMENTRO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DEMENTO ALAMBRE ARENA Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIADO DEMENTO ARENA ALAMBRE Y MADERA": 7,
        lambda df: df['material_pared_exterior_esp'] == "VACIOADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "VACUADO DE CEMENTO Y ARENA": 4,
        lambda df: df['material_pared_exterior_esp'] == "PLAQUETAR PREFABRICADA": 7,
    },

    'material_pared_icv2': { # parevivi
        lambda df: df['material_pared_exterior_icv2'] == 9: 1,
        lambda df: df['material_pared_exterior_icv2'].isin([6, 7]): 2,
        lambda df: df['material_pared_exterior_icv2'].isin([2, 4, 5]): 3,
        'default': 9
    }
}

In [ ]:
select_icv_map = {
    'material_piso_icv2': { # pisovivi
        lambda df: (df['material_piso'] == 9): 0,
        lambda df: (df['material_piso'].isin([2, 3, 6, 8])): 2,
        lambda df: df['material_piso'].isna(): 9,
        'default': 1
    },

    'material_techo_icv2': { # techvivi
        lambda df: (df['material_techo'].isin([1, 4])): 1,
        lambda df: (df['material_techo'] == 2): 2,
        lambda df: df['material_techo'].isna(): 9,
        'default': 0
    },

    'tipo_sanitario_icv2': { # sanivivis2
        lambda df: (df['tipo_sanitario'] == 1) | (df['tipo_sanitario'] == 2): 3,
        lambda df: (df['tipo_sanitario'] == 3) | (df['tipo_sanitario'] == 4): 2,
        lambda df: df['tipo_sanitario'] == 5: 0,
        lambda df: df['tipo_sanitario'].isna(): 9
    },

    'tiene_agua_tuberia_publica_icv1': { # aguavivi
        lambda df: (df['donde_proviene_agua'] == 2): 1,
        lambda df: df['donde_proviene_agua'] == 9: 2,
        lambda df: df['donde_proviene_agua'] == 1: 3,
        lambda df: df['donde_proviene_agua'].isna(): 9,
        'default': 0
    },

    'agua_vivienda_icv0': { # aguavivi0
        lambda df: (df['tiene_agua_red_publica'] == 1): 3,
        lambda df: df['tiene_agua_red_publica'].isna(): 9,
        'default': 0
    },

    'tiene_agua_dentro_patio_icv2': { # aguavivi1, aguavivis2
        lambda df: (df['donde_proviene_agua'] <= 2): 3,
        lambda df: df['donde_proviene_agua'].isna(): 9,
        'default': 0
    },

    'como_elimina_basura_icv1': { # basuvivi
        lambda df: (df['como_elimina_basura'] == 4): 1,
        lambda df: (df['como_elimina_basura'] == 1): 2,
        lambda df: (df['como_elimina_basura'] == 2): 3,
        lambda df: df['como_elimina_basura'].isna(): 9,
        'default': 0
    },

    'como_elimina_basura_icv2': { # basuvivi2
        lambda df: (df['como_elimina_basura_icv1'] == 0): 1,
        lambda df: (df['como_elimina_basura_icv1'] == 1): 0,
        lambda df: (df['como_elimina_basura_icv1'].isin([2, 3])): 2,
        'default': 9
    },

    'situacion_equipos_icv2': { # equivivi
        lambda df: df['cantidad_equipos_basicos'] == 0: 0,
        lambda df: df['cantidad_equipos_basicos'].isin(range(1, 4)): 1,
        lambda df: (df['cantidad_equipos_basicos'] == 4) & (df['cantidad_equipos_modernos'] == 0): 2,
        lambda df: (df['cantidad_equipos_basicos'] == 4) & (df['cantidad_equipos_modernos'].isin([1, 2])): 3,
        lambda df: df['cantidad_equipos_modernos'] == 3: 4,
        lambda df: df['cantidad_equipos_modernos'].isna(): 9,
        'default': 9
    },

    'tipo_alumbrado_icv1': { # luzvivi
        lambda df: df['tipo_alumbrado'] == 1: 1,
        lambda df: df['tipo_alumbrado'].isin([2, 3]): 2,
        lambda df: df['tipo_alumbrado'].isna(): 9,
        'default': 0
    },

    'tipo_alumbrado_icv2': { # luzvivi2
        lambda df: df['tipo_alumbrado'] == 0: 0,
        lambda df: df['tipo_alumbrado'].isin([1, 2]): 1,
        'default': 9
    },

    'combustible_para_cocinar_icv1': { # cocivivi
        lambda df: df['combustible_para_cocinar'].isin([1, 2, 3]): 1,
        lambda df: df['combustible_para_cocinar'] == 6: 2,
        lambda df: df['combustible_para_cocinar'].isna(): 9,
        'default': 0
    },

    'combustible_para_cocinar_icv2': { # cocivivi2
        lambda df: df['combustible_para_cocinar_icv1'] == 0: 0,
        lambda df: df['combustible_para_cocinar_icv1'] == 1: 2,
        lambda df: df['combustible_para_cocinar_icv1'] == 2: 1,
        'default': 9
    },

    'tipo_vivienda_icv2': { # tipovivis22
        lambda df: df['tipo_vivienda'] <= 3: 1,
        lambda df: df['tipo_vivienda'].isin([4, 5]): 2,
        lambda df: df['tipo_vivienda'].isna(): 9,
        'default': 0
    },

    'hacinamiento_icv': { # hacin
        lambda df: df['cant_dormitorios_vivienda'].isna(): 99,
        'default': lambda df: (df['cantidad_miembros'] / df['cant_dormitorios_vivienda'])
    },

    'hacinamiento_icv2': { # hacinadovivi
        lambda df: (4 <= df['hacinamiento_icv']) & (df['hacinamiento_icv'] < 99): 0,
        lambda df: (2.5 <= df['hacinamiento_icv']) & (df['hacinamiento_icv'] < 4): 1,
        lambda df: (2 <= df['hacinamiento_icv']) & (df['hacinamiento_icv'] < 2.5): 2,
        lambda df: df['hacinamiento_icv'] < 2: 3,
        lambda df: df['cant_dormitorios_vivienda'] == 0: 0,
        lambda df: df['cant_dormitorios_vivienda'].isna(): 9,
    },

    'analfabeto_icv1': { # analfa
        lambda df: df['edad'] <= 5: 9,
        lambda df: df['sabe_leer_escribir'] == 2: 1,
        lambda df: df['sabe_leer_escribir'] == 1: 0,
        'default': 9
    },

    'nivel_educativo_icv1': { # niveledu
        lambda df: (1 <= df['nivel_ultimo_ano_aprobado']) & (df['nivel_ultimo_ano_aprobado'] < 6):
            lambda df: df['nivel_ultimo_ano_aprobado'],
        lambda df: (6 <= df['nivel_ultimo_ano_aprobado']) & (df['nivel_ultimo_ano_aprobado'] <= 8): 6,
        lambda df: df['nivel_ultimo_ano_aprobado'].isin([9, 10]): 0,
        lambda df: df['edad'] <= 2: 8,
        'default': 9       
    },

    'años_educacion_icv1': { # anosedu1
        lambda df: df['nivel_educativo_icv1'] <= 1: 0,
        lambda df: df['nivel_educativo_icv1'] == 2: lambda df: df['ultimo_ano_aprobado'],
        lambda df: df['nivel_educativo_icv1'] == 3: lambda df: 8 + df['ultimo_ano_aprobado'],
        lambda df: df['nivel_educativo_icv1'] == 4: lambda df: 8 + df['ultimo_ano_aprobado'],
        lambda df: df['nivel_educativo_icv1'] == 5: lambda df: 12 + df['ultimo_ano_aprobado'],
        lambda df: df['nivel_educativo_icv1'] == 3: lambda df: 8 + df['ultimo_ano_aprobado'],
        lambda df: df['edad'] <= 2: 0,             
    },

    'años_educacion_mayores_15_icv1': { # anosedu15
        lambda df: df['edad'] >= 15: lambda df: df['años_educacion_icv1'],
        lambda df: df['edad'] < 15: 99,
        'default': 99
    },

    'alfabeto_icv1': { # alfabeto
        lambda df: ((df['edad'] < 5) | (df['analfabeto_icv1'] == 9)) & (df['años_educacion_mayores_15_icv1'] == 0): 0,
        lambda df: df['edad'] < 5: 9,
        lambda df: df['analfabeto_icv1'] == 0: 1,
        lambda df: df['analfabeto_icv1'] == 1: 0,
        'default': 9
    },

    'alfabetos_mayores_15_icv2': { # alfa15
        lambda df: df['edad'] < 15: 9,
        lambda df: (df['edad'] >= 15) & (df['sabe_leer_escribir'].isna()): 9,
        'default': 9
    },

    'años_educacion_jefe_icv1': { # educjef
        lambda df: df['parentesco'] == 1: lambda df: df['años_educacion_icv1'],
        'default': 99
    },

    'proporcion_menor_5_icv2': { # pninme5vivi
        lambda df: (0.25 < df['cantidad_menor_5_promedio']) & (df['cantidad_menor_5_promedio'] <= 1): 0,
        lambda df: (0 < df['cantidad_menor_5_promedio']) & (df['cantidad_menor_5_promedio'] <= 0.25): 0,
        lambda df: df['cantidad_menor_5_promedio'] == 0: 2,
        'default': 9
    },

    'inscrito_icv2': { # inscrito
        lambda df: (1 <= df['nivel_se_matriculo']) & (df['nivel_se_matriculo'] <= 8): 1,
        lambda df: (df['nivel_se_matriculo'] == 9) & df['mes'].isin(range(6, 9)) & df['porque_no_estudia'] == 1: 1,
        lambda df: df['edad'] <= 2: 999,
        'default': 0
    },

    'asistencia_escolar_de_6_a_14_icv1': { # asis614
        lambda df: (df['edad'] < 6) | (df['edad'] > 14): 9,
        lambda df: df['inscrito_icv2'] == 0: 0,
        lambda df: df['inscrito_icv2'] == 1: 1,
        'default': 9
    },

    'estratos_icv2': { # ESTRATOS2
        lambda df: (df['id_provincia'] == 1) | ((df['id_provincia'] == 32) & (df['zona'] == 1)) | ((df['id_municipio'] == 2501) | df['zona'] == 1): 1,
        lambda df: df['zona'] == 1: 2,
        lambda df: df['zona'] == 2: 3,
        'default': 9
    }
}

select_wall_material_map.update(select_icv_map)

select_icv_transformer = SelectTransformer(select_wall_material_map)
survey_icv_selected = select_icv_transformer.transform(survey_icv_assigned)

## Assign Transformer

In [ ]:
assign_icv_map_2 = {
    'años_educacion_promedio_icv1': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['alfabetos_mayores_15_icv2'].transform('mean'), # anosedu15h
    'años_educacion_jefe_promedio_icv1': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['años_educacion_jefe_icv1'].transform('mean'), # educjefh
    'alfabetos_mayores_15_promedio_icv1': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['alfabetos_mayores_15_icv2'].transform('mean'), # alfa15h
    'asistencia_escolar_de_6_a_14_promedio_icv1': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['asistencia_escolar_de_6_a_14_icv1'].transform('mean'), # asis614h

    'miembro_de_6_a_14_icv1': lambda df: (6 <= df['edad']) & (df['edad'] <= 14), # pob6a14,
    'miembro_de_6_a_14_hogar_icv1': lambda df: df.groupby(['trimestre', 'vivienda', 'hogar'])['miembro_de_6_a_14_icv1'].transform('sum') # asis614h
}

assign_icv_transformer_2 = AssignTransformer(assign_icv_map_2)
survey_icv_assigned_2 = assign_icv_transformer_2.transform(survey_icv_selected)

## Select Transformer

In [ ]:
select_icv_map_2 = {
    'escolaridad_jefe_media_icv2': { # escojefevivi
        lambda df: df['años_educacion_jefe_promedio_icv1'].isna(): 9,
        lambda df: df['años_educacion_jefe_promedio_icv1'] == 0: 0,
        lambda df: (1 <= df['años_educacion_jefe_promedio_icv1']) & (df['años_educacion_jefe_promedio_icv1'] < 4): 1,
        lambda df: (4 <= df['años_educacion_jefe_promedio_icv1']) & (df['años_educacion_jefe_promedio_icv1'] < 8): 2,
        lambda df: (8 <= df['años_educacion_jefe_promedio_icv1']) & (df['años_educacion_jefe_promedio_icv1'] < 12): 3,
        lambda df: (12 <= df['años_educacion_jefe_promedio_icv1']) & (df['años_educacion_jefe_promedio_icv1'] < 16): 4,
        lambda df: df['años_educacion_jefe_promedio_icv1'] >= 16: 5,
        'default': 0
    },

    'escolaridad_hogar_media_icv2': { # escovivi
        lambda df: df['años_educacion_promedio_icv1'].isna(): 9,
        lambda df: df['años_educacion_promedio_icv1'] < 1: 0,
        lambda df: (1 <= df['años_educacion_promedio_icv1']) & (df['años_educacion_promedio_icv1'] < 4): 1,
        lambda df: (4 <= df['años_educacion_promedio_icv1']) & (df['años_educacion_promedio_icv1'] < 8): 2,
        lambda df: (8 <= df['años_educacion_promedio_icv1']) & (df['años_educacion_promedio_icv1'] < 12): 3,
        lambda df: (12 <= df['años_educacion_promedio_icv1']) & (df['años_educacion_promedio_icv1'] < 16): 4,
        lambda df: df['años_educacion_promedio_icv1'] >= 16: 5,
        'default': 0
    },

    'proporcion_alfabetos_hogar_icv2': { # alfaviviS2
        lambda df: df['alfabetos_mayores_15_promedio_icv1'].isna(): 9,
        lambda df: df['alfabetos_mayores_15_promedio_icv1'] == 0: 0,
        lambda df: (0 < df['alfabetos_mayores_15_promedio_icv1']) & (df['alfabetos_mayores_15_promedio_icv1'] < 0.5): 1,
        lambda df: (0.5 <= df['alfabetos_mayores_15_promedio_icv1']) & (df['alfabetos_mayores_15_promedio_icv1'] < 0.70): 2,
        lambda df: (0.70 <= df['alfabetos_mayores_15_promedio_icv1']) & (df['alfabetos_mayores_15_promedio_icv1'] < 0.99): 3,
        lambda df: df['alfabetos_mayores_15_promedio_icv1'] >= 0.99: 5,
        'default': 0
    },

    'asistencia_escolar_de_6_a_14_icv2': { # asiscolvivi
        lambda df: df['asistencia_escolar_de_6_a_14_promedio_icv1'] == 1: 2,
        lambda df: (0 < df['asistencia_escolar_de_6_a_14_promedio_icv1']) & (df['asistencia_escolar_de_6_a_14_promedio_icv1'] < 1): 1,
        lambda df: df['asistencia_escolar_de_6_a_14_promedio_icv1'] == 0: 0,
        lambda df: df['miembro_de_6_a_14_hogar_icv1'] == 0: 3
    },
}

select_icv_transformer_2 = SelectTransformer(select_icv_map_2)
survey_icv_selected_2 = select_icv_transformer_2.transform(survey_icv_assigned_2)

# Modelos del ICV

## Metropolitano

In [ ]:
select_metropolitano_map = {
    'piso_siuben_icv2' : {
        lambda df: (df['estratos_icv2'] == 1) & (df['material_piso_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_piso_icv2'] == 1): 2.260294,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_piso_icv2'] == 2): 4.723993,
    },

    'pared_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['material_pared_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_pared_icv2'] == 1): 0.374075,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_pared_icv2'] == 2): 0.374075,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_pared_icv2'] == 3): 3.652266,
    },

    'techo_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['material_techo_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_techo_icv2'] == 1): 0.625379,
        lambda df: (df['estratos_icv2'] == 1) & (df['material_techo_icv2'] == 2): 3.282922
    },

    'agua_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['tiene_agua_dentro_patio_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['tiene_agua_dentro_patio_icv2'] == 3): 2.769835
    },

    'equipo_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['situacion_equipos_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['situacion_equipos_icv2'] == 1): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['situacion_equipos_icv2'] == 2): 0.690480,
        lambda df: (df['estratos_icv2'] == 1) & (df['situacion_equipos_icv2'] == 3): 2.242501,
        lambda df: (df['estratos_icv2'] == 1) & (df['situacion_equipos_icv2'] == 4): 3.131370
    },

    'escolaridad_jefe_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_jefe_media_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_jefe_media_icv2'] == 1): 1.755932,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_jefe_media_icv2'] == 2): 2.454759,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_jefe_media_icv2'] == 3): 2.721829,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_jefe_media_icv2'] == 4): 4.115244,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_jefe_media_icv2'] == 5): 5.221036
    },

    'escolaridad_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_hogar_media_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_hogar_media_icv2'] == 1): 1.779626,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_hogar_media_icv2'] == 2): 2.534996,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_hogar_media_icv2'] == 3): 3.722092,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_hogar_media_icv2'] == 4): 5.346769,
        lambda df: (df['estratos_icv2'] == 1) & (df['escolaridad_hogar_media_icv2'] == 5): 6.064893,
    },

    'alfabetos_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_alfabetos_hogar_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_alfabetos_hogar_icv2'] == 1): 0.466132,
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_alfabetos_hogar_icv2'] == 2): 1.457965,
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_alfabetos_hogar_icv2'] == 3): 3.478855,
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_alfabetos_hogar_icv2'] == 4): 4.941860
    },

    'luz_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['tipo_alumbrado_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['tipo_alumbrado_icv2'] == 1): 33.314324
    },

    'sanitario_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['tipo_sanitario_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['tipo_sanitario_icv2'] == 2): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['tipo_sanitario_icv2'] == 3): 5.266163
    },

    'cocina_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['combustible_para_cocinar_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['combustible_para_cocinar_icv2'] == 1): 4.936530,
        lambda df: (df['estratos_icv2'] == 1) & (df['combustible_para_cocinar_icv2'] == 2): 11.196426
    },

    'hacinamiento_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['hacinamiento_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['hacinamiento_icv2'] == 1): 1.516339,
        lambda df: (df['estratos_icv2'] == 1) & (df['hacinamiento_icv2'] == 2): 2.548382,
        lambda df: (df['estratos_icv2'] == 1) & (df['hacinamiento_icv2'] == 3): 3.864894
    },

    'menores_5_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_menor_5_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_menor_5_icv2'] == 1): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['proporcion_menor_5_icv2'] == 2): 2.605800
    },

    'asistencia_escolar_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 1) & (df['asistencia_escolar_de_6_a_14_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 1) & (df['asistencia_escolar_de_6_a_14_icv2'] == 1): 8.722061,
        lambda df: (df['estratos_icv2'] == 1) & (df['asistencia_escolar_de_6_a_14_icv2'] == 2): 8.977462,
        lambda df: (df['estratos_icv2'] == 1) & (df['asistencia_escolar_de_6_a_14_icv2'] == 3): 9.964218
    }
}

select_metropolitano_transformer = SelectTransformer(select_metropolitano_map)
survey_icv_metropolitano_selected = select_metropolitano_transformer.transform(survey_icv_selected_2)

## Resto Urbano

In [ ]:
select_urban_map = {
    'piso_siuben_u_icv2' : {
        lambda df: (df['estratos_icv2'] == 2) & (df['material_piso_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_piso_icv2'] == 1): 3.383589,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_piso_icv2'] == 2): 5.641851,
    },

    'pared_siuben_u_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['material_pared_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_pared_icv2'] == 1): 0.532922,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_pared_icv2'] == 2): 1.128277,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_pared_icv2'] == 3): 3.145826,
    },

    'techo_siuben_u_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['material_techo_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_techo_icv2'] == 1): 1.933012,
        lambda df: (df['estratos_icv2'] == 2) & (df['material_techo_icv2'] == 2): 4.264114,
    },

    'agua_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['tiene_agua_dentro_patio_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['tiene_agua_dentro_patio_icv2'] == 3): 2.084813,
    },

    'sanitario_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_sanitario_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_sanitario_icv2'] == 2): 1.150724,
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_sanitario_icv2'] == 3): 3.526838,
    },

    'equipo_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['situacion_equipos_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['situacion_equipos_icv2'] == 1): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['situacion_equipos_icv2'] == 2): 1.108713,
        lambda df: (df['estratos_icv2'] == 2) & (df['situacion_equipos_icv2'] == 3): 2.292429,
        lambda df: (df['estratos_icv2'] == 2) & (df['situacion_equipos_icv2'] == 4): 3.166067,
    },
    
    'provincia_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 2): 1.053866,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 3): 0.494673,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 4): 0.928930,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 5): 1.179354,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 6): 2.213650,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 7): 0.486756,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 8): 1.535589,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 9): 2.244524,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 10): 0.065853,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 11): 2.372389,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 12): 2.137986,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 13): 2.856198,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 14): 2.081003,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 15): 0.604124,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 16): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 17): 1.618325,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 18): 2.574898,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 19): 2.240384,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 20): 2.422620,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 21): 2.171819,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 22): 1.506214,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 23): 2.222667,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 24): 1.864328,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 25): 1.939559,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 26): 1.923409,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 27): 0.952579,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 28): 2.886196,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 29): 0.861634,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 30): 1.238621,
        lambda df: (df['estratos_icv2'] == 2) & (df['id_provincia'] == 31): 1.235454,
    },

    'escolaridad_jefe_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_jefe_media_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_jefe_media_icv2'] == 1): 2.304242,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_jefe_media_icv2'] == 2): 2.843586,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_jefe_media_icv2'] == 3): 3.562974,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_jefe_media_icv2'] == 4): 4.465421,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_jefe_media_icv2'] == 5): 5.645793,
    },

    'escolaridad_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_hogar_media_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_hogar_media_icv2'] == 1): 2.522623,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_hogar_media_icv2'] == 2): 3.869082,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_hogar_media_icv2'] == 3): 4.991055,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_hogar_media_icv2'] == 4): 6.153842,
        lambda df: (df['estratos_icv2'] == 2) & (df['escolaridad_hogar_media_icv2'] == 5): 7.304393,
    },

    'alfabetos_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['proporcion_alfabetos_hogar_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['proporcion_alfabetos_hogar_icv2'] == 1): 1.165989,
        lambda df: (df['estratos_icv2'] == 2) & (df['proporcion_alfabetos_hogar_icv2'] == 2): 2.905762,
        lambda df: (df['estratos_icv2'] == 2) & (df['proporcion_alfabetos_hogar_icv2'] == 3): 4.120705,
        lambda df: (df['estratos_icv2'] == 2) & (df['proporcion_alfabetos_hogar_icv2'] == 4): 5.535506,
    },

    'luz_siuben_u_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_alumbrado_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_alumbrado_icv2'] == 1): 17.818472,
    },

    'basura_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['como_elimina_basura_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['como_elimina_basura_icv2'] == 1): 2.380818,
        lambda df: (df['estratos_icv2'] == 2) & (df['como_elimina_basura_icv2'] == 2): 7.416184,
        },

    'cocina_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['combustible_para_cocinar_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['combustible_para_cocinar_icv2'] == 1): 3.442112,
        lambda df: (df['estratos_icv2'] == 2) & (df['combustible_para_cocinar_icv2'] == 2): 8.786023,
    },

    'tipo_vivienda_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_vivienda_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_vivienda_icv2'] == 1): 6.263855,
        lambda df: (df['estratos_icv2'] == 2) & (df['tipo_vivienda_icv2'] == 2): 16.332073,
    },

    'hacinamiento_u_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 2) & (df['hacinamiento_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 2) & (df['hacinamiento_icv2'] == 1): 3.588715,
        lambda df: (df['estratos_icv2'] == 2) & (df['hacinamiento_icv2'] == 2): 5.117803,
        lambda df: (df['estratos_icv2'] == 2) & (df['hacinamiento_icv2'] == 3): 6.445851,
    },
}

select_urban_transformer = SelectTransformer(select_urban_map)
survey_icv_urban_selected = select_urban_transformer.transform(survey_icv_metropolitano_selected)

## Rural

In [ ]:
select_rural_map = {
    'luz_siuben_r_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_alumbrado_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_alumbrado_icv2'] == 1): 4.718194,
    },

    'agua_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['tiene_agua_dentro_patio_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['tiene_agua_dentro_patio_icv2'] == 3): 2.452326,
    },

    'basura_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['como_elimina_basura_icv2'] == 0): 0.617003,
        lambda df: (df['estratos_icv2'] == 3) & (df['como_elimina_basura_icv2'] == 1): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['como_elimina_basura_icv2'] == 2): 2.776191,
    },

    'cocina_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['combustible_para_cocinar_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['combustible_para_cocinar_icv2'] == 1): 0.656332,
        lambda df: (df['estratos_icv2'] == 3) & (df['combustible_para_cocinar_icv2'] == 2): 3.408643,
    },

    'equipos_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['situacion_equipos_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['situacion_equipos_icv2'] == 1): 2.050546,
        lambda df: (df['estratos_icv2'] == 3) & (df['situacion_equipos_icv2'] == 2): 3.611402,
        lambda df: (df['estratos_icv2'] == 3) & (df['situacion_equipos_icv2'] == 3): 5.050184,
        lambda df: (df['estratos_icv2'] == 3) & (df['situacion_equipos_icv2'] == 4): 6.558594,
    },

    'provincia_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 2): 2.337846,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 3): 0.541928,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 4): 1.172178,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 5): 2.235231,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 6): 3.089057,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 7): 0.056092,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 8): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 9): 3.197998,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 10): 1.254654,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 11): 2.188282,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 12): 1.387518,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 13): 3.958634,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 14): 3.244731,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 15): 1.821101,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 16): 0.505030,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 17): 3.491828,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 18): 3.001598,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 19): 3.246290,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 20): 3.294250,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 21): 3.346544,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 22): 1.501330,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 23): 1.977301,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 24): 3.110951,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 25): 4.807590,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 26): 2.361920,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 27): 1.182542,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 28): 4.729503,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 29): 1.206339,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 30): 0.294667,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 31): 0.774227,
        lambda df: (df['estratos_icv2'] == 3) & (df['id_provincia'] == 32): 4.167915,
    },

    'escolaridad_jefe_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_jefe_media_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_jefe_media_icv2'] == 1): 2.085314,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_jefe_media_icv2'] == 2): 2.683586,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_jefe_media_icv2'] == 3): 3.563038,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_jefe_media_icv2'] == 4): 4.851029,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_jefe_media_icv2'] == 5): 5.810809,
    },

    'escolaridad_hogar_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_hogar_media_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_hogar_media_icv2'] == 1): 2.184700,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_hogar_media_icv2'] == 2): 3.486579,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_hogar_media_icv2'] == 3): 4.791990,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_hogar_media_icv2'] == 4): 6.140607,
        lambda df: (df['estratos_icv2'] == 3) & (df['escolaridad_hogar_media_icv2'] == 5): 6.550803,

        lambda df: (df['estratos_icv2'] == 3) & (df['proporcion_alfabetos_hogar_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['proporcion_alfabetos_hogar_icv2'] == 1): 1.869298,
        lambda df: (df['estratos_icv2'] == 3) & (df['proporcion_alfabetos_hogar_icv2'] == 2): 2.540936,
        lambda df: (df['estratos_icv2'] == 3) & (df['proporcion_alfabetos_hogar_icv2'] == 3): 3.468555,
        lambda df: (df['estratos_icv2'] == 3) & (df['proporcion_alfabetos_hogar_icv2'] == 4): 4.768940,
    },

    'piso_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['material_piso_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_piso_icv2'] == 1): 4.588612,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_piso_icv2'] == 2): 8.438427,
    },

    'pared_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['material_pared_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_pared_icv2'] == 1): 0.851715,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_pared_icv2'] == 2): 2.380679,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_pared_icv2'] == 3): 4.943913,
    },

    'techo_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['material_techo_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_techo_icv2'] == 1): 5.800419,
        lambda df: (df['estratos_icv2'] == 3) & (df['material_techo_icv2'] == 2): 9.934105,
    },

    'sanitario_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_sanitario_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_sanitario_icv2'] == 2): 2.162967,
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_sanitario_icv2'] == 3): 5.136840,
    },

    'tipo_vivienda_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_vivienda_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_vivienda_icv2'] == 1): 10.798609,
        lambda df: (df['estratos_icv2'] == 3) & (df['tipo_vivienda_icv2'] == 2): 23.209360,
    },

    'hacinamiento_r_siuben_icv2': {
        lambda df: (df['estratos_icv2'] == 3) & (df['hacinamiento_icv2'] == 0): 0,
        lambda df: (df['estratos_icv2'] == 3) & (df['hacinamiento_icv2'] == 1): 5.275550,
        lambda df: (df['estratos_icv2'] == 3) & (df['hacinamiento_icv2'] == 2): 5.844578,
        lambda df: (df['estratos_icv2'] == 3) & (df['hacinamiento_icv2'] == 3): 6.485263,
    },
}

select_rural_transformer = SelectTransformer(select_rural_map)
survey_icv_rural_selected = select_rural_transformer.transform(survey_icv_urban_selected)

## Calculate ICV

In [ ]:
select_set_icv_map = {
    'icv': {
        lambda df: df['estratos_icv2'] == 1: lambda df, cols=select_metropolitano_map.keys(): df[cols].sum(1),
        lambda df: df['estratos_icv2'] == 2: lambda df, cols=select_urban_map.keys(): df[cols].sum(1),
        lambda df: df['estratos_icv2'] == 3: lambda df, cols=select_rural_map.keys(): df[cols].sum(1)
    },

    'icv_s1': {
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] <= 64.56): 1,
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] > 64.56 ) & (df['icv'] <= 79.24): 2,
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] > 79.24) & (df['icv'] <= 91.45): 3,
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] > 91.45): 4,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] <= 56.55): 1,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] > 56.55) & (df['icv'] <= 69.07): 2,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] > 69.07) & (df['icv'] <= 81.19): 3,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] > 81.19): 4,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv'] <= 43.62): 1,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv'] >   43.62) & (df['icv'] <= 58.07): 2,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv'] >   58.07) & (df['icv'] <= 73.95): 3,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv'] >   73.95): 4,
    },

    'icv_basico': {
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] <= 71.80): 1,
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] > 71.80) & (df['icv'] <= 83.07): 2,
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] > 83.07) & (df['icv'] <= 91.13): 3,
        lambda df: (df['estratos_icv2'] == 1) & (df['icv'] > 91.13): 4,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] <= 57.48): 1,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] > 57.48) & (df['icv'] <= 71.72): 2,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] > 71.72) & (df['icv'] <= 80.87): 3,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] > 80.87): 4,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv'] <= 42.64): 1,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >  42.64) & (df['icv'] <= 56.88): 2,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >  56.88) & (df['icv'] <= 72.14): 3,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >  72.14): 4,
    },

    'icv_decilico': {
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  < 77.0107): 1,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 77.0107) & (df['icv']  < 81.5188): 2,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 81.5188) & (df['icv']  < 84.2895): 3,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 84.2895) & (df['icv']  < 86.7631): 4,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 86.7631) & (df['icv']  < 89.2352): 5,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 89.2352) & (df['icv']  < 91.3963): 6,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 91.3963) & (df['icv']  < 93.2715): 7,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 93.2715) & (df['icv']  < 95.0447): 8,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 95.0447) & (df['icv']  < 97.2872): 9,
        lambda df: (df['estratos_icv2'] == 1) & ( df['icv']  >= 97.2872): 10,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] < 60.2126): 1,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 60.2126) & (df['icv'] < 66.6781): 2,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 66.6781) & (df['icv'] < 70.3410): 3,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 70.3410) & (df['icv'] < 72.9504): 4,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 72.9504) & (df['icv'] < 75.1317): 5,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 75.1317) & (df['icv'] < 77.0244): 6,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 77.0244) & (df['icv'] < 79.2092): 7,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 79.2092) & (df['icv'] < 81.5364): 8,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 81.5364) & (df['icv'] < 84.7953): 9,
        lambda df: (df['estratos_icv2'] == 2) & (df['icv'] >= 84.7953): 10,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  < 41.4199): 1,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 41.4199) & (df['icv']  < 48.7159): 2,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 48.7159) & (df['icv']  < 53.1760): 3,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 53.1760) & (df['icv']  < 56.6559): 4,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 56.6559) & (df['icv']  < 59.4876): 5,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 59.4876) & (df['icv']  < 62.5849): 6,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 62.5849) & (df['icv']  < 65.9946): 7,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 65.9946) & (df['icv']  < 69.3230): 8,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >= 69.3230) & (df['icv']  < 74.1889): 9,
        lambda df: (df['estratos_icv2'] == 3) & (df['icv']  >=  74.1889): 10
    },

    'icv_basico_cat': {
        lambda df: df['icv_basico'] == 1: 'Pobreza extrema',
        lambda df: df['icv_basico'] == 2: 'Pobreza moderada',
        lambda df: df['icv_basico'] == 3: 'Estrato medio',
        lambda df: df['icv_basico'] == 4: 'Estrato alto',
    }
}

select_set_icv_transformer = SelectTransformer(select_set_icv_map)
survey_set_icv_selected = select_set_icv_transformer.transform(survey_icv_rural_selected)

In [ ]:
survey_set_icv_selected[['icv', 'icv_basico_cat']]

# Aggregation Transformer

In [ ]:
aggregation_icv_map = {
    'mean': ['icv']
}

keys = ['trimestre', 'id_hogar']

first = ['icv', 'icv_basico', 'icv_basico_cat', 'factor_expansion']

aggregation_icv_transformer = AggregateTransformer(aggregation_icv_map, groupby_=keys, keep_first=first)
survey_icv_household_aggregated = aggregation_icv_transformer.transform(survey_set_icv_selected)

In [ ]:
survey_icv_household_aggregated.columns

# Save dataset

In [ ]:
save_survey_with_pickle(survey_icv_household_aggregated, ENCFT_OBJECT_DIR / 'encft-2020-icv-household.pkl')

# Checks

In [ ]:
survey_icv_household_aggregated['fa'] = survey_icv_household_aggregated['factor_expansion_first'] / 4

In [ ]:
get_percentage_table_with_expansion_factor(survey_icv_household_aggregated, 'icv_basico_cat_first', 'fa')

In [ ]:
survey_set_icv_selected['icv_basico_cat'].value_counts()

In [ ]:
len('Pobreza moderada')